In [1]:
%load_ext autoreload
%autoreload 2

import gc
import cv2
import PIL
import random
import numpy as np
import os
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pdb
from dataclasses import dataclass
from functools import partial

from skimage.morphology import label
from sklearn.model_selection import KFold

from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.nn.functional import binary_cross_entropy_with_logits
import torch
import torch.nn as nn
from torch.utils.data import WeightedRandomSampler, Sampler

from fastai.vision.data import SegmentationItemList, SegmentationLabelList, ImageList, imagenet_stats
from fastai.data_block import FloatList, FloatItem
from fastai.basic_data import DatasetType
from fastai.basic_train import Learner
from fastai.callback import OptimWrapper
from fastai.vision.image import Image, ImageSegment, image2np, pil2tensor
from fastai.vision.transform import get_transforms
from fastai.vision.learner import unet_learner, cnn_learner
import fastai.vision.models as mod
from fastai.callbacks import SaveModelCallback, LearnerCallback
from fastai.metrics import accuracy

from pathlib import Path

# IMAGE SIZES
TRAIN_SIZE = 256
MAX_SIZE = 1388
TEST_SIZE = 224
TEST_OVERLAP = 64
IMG_CHANNELS = 3

# PATHS
PROJECT_PATH = Path(
    '/work/stages/schwob/siim-pneumothorax')
FULL_TRAIN_PATH = PROJECT_PATH/'data/dicom-images-train'
FULL_TEST_PATH = PROJECT_PATH/'data/dicom-images-test'
DATA = PROJECT_PATH/'data'
TRAIN_PATH = PROJECT_PATH/'data/train'
TEST_PATH = PROJECT_PATH/'data/test'
MODELS_PATH = PROJECT_PATH/'models/'
SUB_PATH = PROJECT_PATH/'submissions/'
LABELS_OLD = PROJECT_PATH/'data/train-rle.csv'
LABELS = PROJECT_PATH/'data/train-rle-fastai2.csv'
LABELS_POS = PROJECT_PATH/'data/train-rle-fastai_pos.csv'
LABELS_CLASSIF = PROJECT_PATH/'data/train-rle-fastai-classif.csv'
LOG = Path('/work/stages/schwob/runs')

# LEARNER CONFIG
BATCH_SIZE = 16
WD = 0.1
LR = 2e-4
GROUP_LIMITS = None
FREEZE_UNTIL = None
EPOCHS = 10
UNFROZE_EPOCHS = 10
PRETRAINED = True
MODEL = 'resnet34'
CLASSES = ['pneum']
ACT = 'sigmoid'

In [14]:
def absol2relat(rle):
    if str(rle) == '-1': return '-1'
    pixels = rle.split()
    new_rle = []
    cur = 0
    for k in range(0, len(pixels), 2):
        if k==0:
            new_rle.append(pixels[k])
            new_rle.append(pixels[k+1])
        else:
            cur = int(pixels[k])
            prev = int(pixels[k-2])+int(pixels[k-1])
            new_rle.append(str(cur-prev))
            new_rle.append(pixels[k+1])
    return ' '.join(new_rle)

In [15]:
def relat2absol(rle):
    if str(rle) == '-1': return '-1'
    pixels = rle.split()
    new_rle = []
    cur = 0
    for k in range(0, len(pixels), 2):
        pix = pixels[k]
        cur += int(pix)
        length = pixels[k+1]
        new_rle.append(str(cur))
        new_rle.append(length)
        cur += int(length)
    return ' '.join(new_rle)

In [8]:
def merge_rles(rle1, rle2):
    if rle1 == rle2: return rle1
    i1 = 0
    i2 = 0
    rle = []
    pixels1 = relat2absol(rle1).split()
    pixels2 = relat2absol(rle2).split()
    while i1<len(pixels1) and i2<len(pixels2):
        p1 = int(pixels1[i1])
        l1 = int(pixels1[i1+1])
        p2 = int(pixels2[i2])
        l2 = int(pixels2[i2+1])
        if p1<=p2: 
            rle.append(str(p1))
            if p2<=p1+l1-1:
                rle.append(str(max(p2-p1+l2, l1)))
                i2 += 2
            else:
                rle.append(str(l1))
            i1 += 2
        else: 
            rle.append(str(p2))
            if p1<=p2+l2-1:
                rle.append(str(max(p1-p2+l1, l2)))
                i1 += 2
            else:
                rle.append(str(l2))
            i2 += 2
            
    rle += pixels1[i1:]+pixels2[i2:]
    return absol2relat(' '.join(rle))

In [9]:
def merge_doubles(old, new):
    df = pd.read_csv(old)
    new_df = pd.DataFrame(columns=['ImageId', 'EncodedPixels'])
    for k, id in enumerate(df['ImageId'].unique()):
        new_rle = ''
        for rle in df.loc[df['ImageId']==id, 'EncodedPixels']:
            new_rle = merge_rles(new_rle, rle)
        new_df.loc[k] = [id, new_rle]
    new_df.to_csv(new, index=False)

In [16]:
for fn in SUB_PATH.iterdir():
    if fn.suffix == '.csv':
        merge_doubles(fn, fn)